In [1]:
import pickle
import json

In [2]:
with open("/home/liuxiao/TuringGithubCapTuring/imagecaption/MturkExps/image_captioning_task/Data/main_caption_data.json",'r') as f:
    caption_d_legacy = json.load(f)

### Compile rebuttal data

In [29]:
filepaths = {
    "chatgpt": "/home/liuxiao/TuringGithub/XiaoData/all_results/chatgpt_image_captioning_v3.pkl",
    "flamingo": "/home/liuxiao/TuringGithub/XiaoData/all_results/flamingo-4B-vitl-rpj3b-langinstruct_caption.pkl",
    "llava": "/home/liuxiao/TuringGithub/XiaoData/all_results/llava-v1.6-mistral-7b_image_captioning_v2.pkl",
    "scst": "/home/liuxiao/TuringGithub/XiaoData/all_results/scst.json"
}
caption_d = {k:{"human":v["human"], "machine":{}} for k, v in caption_d_legacy.items()}
trial_cb_raw = {"human":[],"machine":{}}

for k,v in caption_d_legacy.items():
    for entry in v['human']:
        trial_cb_raw["human"].append([k,entry[0] if entry[0][-1] != '.' else entry[0][:-1],'human',entry[1],''])
    
for method, fp in filepaths.items():
    if method == 'scst':
        with open(fp,'r') as f:
            d = json.load(f)
            for entry in d:
                fn = entry['file_name'].split('/')[-1]
                cap = entry["caption"].strip()
                cap = cap[0].lower() + cap[1:]
                cap = cap if cap[-1] != '.' else cap[:-1]
                caption_d[fn]["machine"][method] = cap
                trial_cb_raw["machine"].setdefault(method,[]).append([fn,cap,"machine",'',method])
    else:
        with open(fp,'rb') as f:
            d = pickle.load(f)
            for fn,cap in d.items():
                cap = cap.strip()
                cap = cap[0].lower() + cap[1:]
                caption_d[fn]["machine"][method] = cap
                cap = cap if cap[-1] != '.' else cap[:-1]
                if method == "flamingo" and "this is a" in cap:
                    cap = cap.replace("this is a picture of ","").replace("this is a photo of ","").replace("this is ","")
                elif method == "flamingo" and "the scene is" in cap:
                    cap = cap.replace("the scene is ","")
                elif method=="flamingo" and "the pitcure is of" in cap:
                    cap = cap.replace("the pitcure is of ","")
                elif method=="flamingo" and "Question:" in cap:
                    cap = cap.split("Question:")[0].strip()
                trial_cb_raw["machine"].setdefault(method,[]).append([fn,cap,"machine",'',method])
                
with open('/home/liuxiao/TuringGithub/XiaoData/all_results/main_caption_data_rebuttal.json','w') as f:
    json.dump(caption_d, f)

### assign counterbalance

In [30]:
with open('/home/liuxiao/TuringGithubCapTuring/imagecaption/MturkExps/image_captioning_task/Turing_Test_with_training/static/data/trials_counterbalanced.json','r') as f:
    trial_cb_legacy = json.load(f)

In [31]:
import random
from copy import deepcopy

trial_cb_raw_copy = deepcopy(trial_cb_raw)
trial_cb_raw_human_copy = deepcopy(trial_cb_raw_copy['human'])
random.seed(1)
random.shuffle(trial_cb_raw_human_copy)
trial_cb_raw_human_copy2 = deepcopy(trial_cb_raw_copy['human'])
random.seed(0)
random.shuffle(trial_cb_raw_human_copy2)
trial_cb_raw_human = trial_cb_raw_human_copy + trial_cb_raw_human_copy2
assert len(trial_cb_raw_human) == 4000

nTotalmachine = 0
for k in trial_cb_raw_copy['machine'].keys():
    random.shuffle(trial_cb_raw_copy['machine'][k])
    nTotalmachine+=len(trial_cb_raw_copy['machine'][k])

machine_caps_interleaved =[] 
for i in range(1000):
    for k, entries in trial_cb_raw_copy['machine'].items():
        machine_caps_interleaved.append(entries[i])

In [32]:
trials_models

[['0964.jpg',
  'a golden waffle is topped with fresh blueberries and powdered sugar',
  'machine',
  '',
  'chatgpt'],
 ['0151.jpg',
  'a kitchen with a stove, a refrigerator, and a microwave',
  'machine',
  '',
  'flamingo'],
 ['0663.jpg',
  'the coin features a portrait of a man wearing glasses, with text in a foreign language around the edge',
  'machine',
  '',
  'llava'],
 ['0060.jpg',
  'a little girl sitting at a table with a birthday cake',
  'machine',
  '',
  'scst'],
 ['0517.jpg',
  'a woman lifts a large tire during outdoor exercise',
  'machine',
  '',
  'chatgpt'],
 ['0539.jpg', 'a girl is sitting in a pink car', 'machine', '', 'flamingo'],
 ['0036.jpg',
  'large white and green airplane on runway',
  'machine',
  '',
  'llava'],
 ['0422.jpg',
  'a woman standing on a street with a car',
  'machine',
  '',
  'scst'],
 ['0454.jpg',
  'a modern train is stationed at a platform',
  'machine',
  '',
  'chatgpt'],
 ['0776.jpg', 'a fish, a tomato, and a potato', 'machine', ''

In [33]:
import random
from collections import Counter

trial_cb_rebuttal = {}
model_dist = {}
nSubject = 100
for i in range(nSubject): 
    trials = trial_cb_raw_human[i*40:i*40+40] 
    assert len(trials) == 40
    trials_models = machine_caps_interleaved[i*40:i*40+40]
    print([_ for _ in Counter([_[-1] for _ in trials_models]).most_common() if _[1] < 20])
    trials += trials_models
    random.shuffle(trials)
    assert len(trials) == 80
    print([_ for _ in Counter([_[0] for _ in trials]).most_common() if _[1] > 1])
    trial_cb_rebuttal[str(i+1)] = trials
        

[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0093.jpg', 2), ('0043.jpg', 2), ('0224.jpg', 2), ('0124.jpg', 2), ('0152.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0371.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0135.jpg', 2), ('0095.jpg', 2), ('0701.jpg', 2), ('0147.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0434.jpg', 2), ('0386.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0887.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0337.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0085.jpg', 3), ('0703.jpg', 2), ('0097.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0570.jpg', 2), ('0616.jpg', 2)]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[]
[('chatgpt', 10), ('flamingo', 10), ('llava', 10), ('scst', 10)]
[('0367.jpg', 2)

In [12]:
len(trials), len(trial_cb_rebuttal)

(80, 100)

In [34]:
with open('/home/liuxiao/TuringGithub/XiaoData/all_results/trials_counterbalanced_rebuttal.json','w') as f:
    json.dump(trial_cb_rebuttal, f)
with open('/home/liuxiao/TuringGithubCapTuring/imagecaption/MturkExps/image_captioning_task/Turing_New_Cap_Models/static/data/trials_counterbalanced_rebuttal.json','w') as f:
    json.dump(trial_cb_rebuttal, f)